# Hierarchical model

* Runs hierarchical model simulations
* Creates dataframes that support the figures.

In [1]:
import os.path as op
from scipy import stats, linalg
import numpy as np
import pandas as pd
import itertools
import utils

## Set up

Assign project paths and variables

In [2]:
proj_dir = '/Volumes/server/Projects/Spatial_Memory/public/osf'
data_dir = op.join(proj_dir, 'data')
df_dir = op.join(data_dir, 'dataframes')

## Hierarchical Model

Set model parameters

In [3]:
filter_sigmas = [5, 15, 30, 45]
stim_sizes = [15, 30, 45, 60]
n_layers = np.arange(4, 11, 2)

stim_amp = 1
x  = np.arange(-180,180,.5); #degrees

Run model for each combination of filter size, stim size, and number of layers

In [4]:
hier_mod = []
for sigma, size, nlay in itertools.product(filter_sigmas, stim_sizes, n_layers):

    # Make gaussian filter
    gauss = stats.norm(0, sigma)
    g = gauss.pdf(x) 
    g = g/np.sum(g)

    # Convert filter to Toeplitz matrix
    W = linalg.toeplitz(np.roll(g, int((len(x)+1)/2)));

    # Make stimulus
    S = np.zeros(x.shape)
    stim_min = int(np.argwhere(x == -size/2))
    stim_max = int(np.argwhere(x == size/2)) + 1
    S[stim_min:stim_max] = stim_amp

    # Start with stimulus and loop over each layer, multiplying each with Toeplitz matrix 
    # to get next layer (equivalent to convolution)
    R = [S];
    for ii in np.arange(nlay):
        R.append(np.dot(W, R[ii]))  
    R = np.vstack(R).T  

    # Now start with the final perception response and run backwards
    Rrev = [R[:, -1]]
    for ii in np.arange(nlay-1):
        Rrev.append(np.dot(W.T, Rrev[ii]))
    Rrev = np.vstack(Rrev).T
    
    # Make forward and reverse dataframes
    df_forward = pd.DataFrame(R[:, 1:], columns=np.arange(1, nlay+1))
    df_forward = df_forward.assign(model='forward', theta=x, n_layers=nlay, filter_sigma=sigma, 
                                   stim_size=size, stim_amp=stim_amp)
    df_reverse = pd.DataFrame(Rrev, columns=np.arange(nlay, 0, -1))
    df_reverse = df_reverse.assign(model='reverse', theta=x, n_layers=nlay, filter_sigma=sigma,
                                   stim_size=size, stim_amp=stim_amp)
    
    # Combine dataframes
    df_mod = pd.concat([df_forward, df_reverse], sort=True).reset_index(drop=True)
    df_mod = df_mod.melt(id_vars=['theta', 'model', 'n_layers', 'filter_sigma', 'stim_size', 'stim_amp'], 
                         var_name='layer', value_name='activity')
    df_mod['activity'] = df_mod['activity'].transform(lambda x: x / df_mod['activity'].max()) #normalize response
    
    hier_mod.append(df_mod)

hier_mod = pd.concat(hier_mod).reset_index(drop=True)

Fit difference of vonmises to each layer in each simulation 

In [5]:
mod_params = utils.fit_diff_vonmises(hier_mod, 'activity', xvar='theta', 
                                     group_cols=['n_layers', 'filter_sigma', 'stim_size', 
                                                 'stim_amp', 'model', 'layer'], 
                                     drop_cols=['hemi', 'roi', 'task'])

Write out model output and fits

In [6]:
hier_mod.to_csv(op.join(df_dir, 'hierarchical_output.csv'), index=False)
mod_params.to_csv(op.join(df_dir, 'hierarchical_fits.csv'), index=False)